# MARKET SALES

[kaggle](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales)

🔘 Problem :Supermarket owners want to predict the prices of goods and merchandise in order to know which branches make more profits and which ones should be developed in order to determine more profit.

🔘 Solving the problem : With the development we are witnessing from artificial intelligence, machine learning models can be used and then trained on a set of training data, then tested on a set of test data, and predicts prices and incomes based on the data to be entered.

### Import Libraries

In [90]:
# basic import
import numpy as np
import pandas as pd

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn import set_config

# split data into training and testing sets
from sklearn.model_selection import train_test_split

# modeling
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# type of training
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold

# metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# tuning parameters
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# save model
import joblib

# others
import warnings
warnings.filterwarnings('ignore')


### Load dataset

In [3]:
df = pd.read_csv("../0.dataset/supermarket_sales.csv")
df.head(10)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,18:30,Ewallet,597.73,4.761905,29.8865,4.1
6,355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.6520,433.6920,2/25/2019,14:36,Ewallet,413.04,4.761905,20.6520,5.8
7,315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.56,10,36.7800,772.3800,2/24/2019,11:38,Ewallet,735.60,4.761905,36.7800,8.0
8,665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.6260,76.1460,1/10/2019,17:15,Credit card,72.52,4.761905,3.6260,7.2
9,692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.84,3,8.2260,172.7460,2/20/2019,13:27,Credit card,164.52,4.761905,8.2260,5.9


#### Pre - EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [6]:
df.describe()

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


### EDA

In [18]:
df["City"].unique()

array(['Yangon', 'Naypyitaw', 'Mandalay'], dtype=object)

In [19]:
df["City"].value_counts()

Yangon       340
Mandalay     332
Naypyitaw    328
Name: City, dtype: int64

### PREPROCESSING

* Check missing values

In [21]:
df.isna().sum()

Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64

* Check duplicates

In [22]:
df.duplicated().sum()

0

* Drop some columns

In [24]:
df = df.drop(columns=["Invoice ID","Date","Time"], axis=1)

* Check the number of uniques values of each column

In [28]:
# define numerical and categorical features
num_features = list(df.select_dtypes(exclude=[object]).columns)
cat_features = list(df.select_dtypes(include=[object]).columns)

print(f"There are {len(num_features)} numerical features: {num_features}")
print(f"There are {len(cat_features)} categorical features: {cat_features}")

There are 8 numerical features: ['Unit price', 'Quantity', 'Tax 5%', 'Total', 'cogs', 'gross margin percentage', 'gross income', 'Rating']
There are 6 categorical features: ['Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Payment']


In [58]:
df.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Payment,cogs,gross margin percentage,gross income,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,Ewallet,522.83,4.761905,26.1415,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,Cash,76.40,4.761905,3.8200,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,Credit card,324.31,4.761905,16.2155,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,Ewallet,465.76,4.761905,23.2880,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,Ewallet,604.17,4.761905,30.2085,5.3


* Split Data -> X and y

In [59]:
# Split X and Y variables
X = df.drop(columns=["Total"], axis=1)
y = df["Total"]

* Create preprocessing pipeline

In [60]:
# Retrieve categorical and numerical columns names
numerical_columns = list(X.select_dtypes(exclude=[object]).columns)
categorical_columns = list(X.select_dtypes(include=[object]).columns)

In [127]:
# Define Pipeline

# for numerical columns
num_pipeline = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False))
    ]
)

# for categorical columns
cat_pipeline = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoder", OneHotEncoder()), # (drop=binary)
        ("scaler", StandardScaler(with_mean=False)) # (with_mean=False)
    ]
)

# Combination of pipelines
preprocessor = ColumnTransformer(
    [
        ("num_pipeline", num_pipeline, numerical_columns),
        ("cat_pipelines", cat_pipeline, categorical_columns)
    ]
)


* Show structure of pipelines

In [72]:
set_config(display="diagram")
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler',
                                                  StandardScaler(with_mean=False))]),
                                 ['Unit price', 'Quantity', 'Tax 5%', 'cogs',
                                  'gross margin percentage', 'gross income',
                                  'Rating']),
                                ('cat_pipelines',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder()),
                                                 ('scaler',
                                                  StandardScaler(with_mean=False))]),
                                 ['Branch', 'City', 'Customer type', 'Gender',
                                  'Product line', 'Payment'])])

* Applying preprocessing

In [ ]:
# Preprocess the data using the pipeline
preprocessed_data = preprocessor.fit_transform(X)
preprocessed_data

In [143]:
# ----------------------------------------------------------------
# Save the preprocessor object
joblib.dump(preprocessor, '../0.dataset/preprocessor.joblib') # preprocessor.pkl

# Load the preprocessor object
preprocessor = joblib.load('../0.dataset/preprocessor.joblib') # preprocessor.pkl



# ----------------------------------------------------------------
# other option
import pickle
file_path = "../0.dataset/"
file_obj = "preprocessor.pkl"

# save
with open(file_path + file_name, "wb") as file_obj_out:
    pickle.dump(preprocessor, file_obj_out)
# load
with open(file_path + file_name, "rb") as file_obj_out:
    preprocessor = pickle.load(file_obj_out)

In [80]:
# ----------------------------------------------------------------
# optional

# Get the column names of the preprocessed data
cat_encoder = preprocessor.named_transformers_['cat_pipelines']['one_hot_encoder']
cat_columns = cat_encoder.get_feature_names_out(categorical_columns)
all_columns = numerical_columns + list(cat_columns)

# Convert the preprocessed data to a dataframe
preprocessed_df = pd.DataFrame(preprocessed_data, columns=all_columns)

# Print the preprocessed dataframe
preprocessed_df.head()

,Unit price,Quantity,Tax 5%,cogs,gross margin percentage,gross income,Rating,Branch_A,Branch_B,Branch_C,...,Gender_Male,Product line_Electronic accessories,Product line_Fashion accessories,Product line_Food and beverages,Product line_Health and beauty,Product line_Home and lifestyle,Product line_Sports and travel,Payment_Cash,Payment_Credit card,Payment_Ewallet
0,2.820473,2.395645,2.233749,2.233749,4.761905,2.233749,5.297718,2.111002,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,2.785353,0.000000,0.000000,0.000000,0.000000,2.103632
1,0.577009,1.711175,0.326413,0.326413,4.761905,0.326413,5.588801,0.000000,0.0,2.129994,...,0.000000,2.662174,0.0,0.0,0.000000,0.000000,0.000000,2.105081,0.000000,0.000000
2,1.749531,2.395645,1.385588,1.385588,4.761905,1.385588,4.308034,2.111002,0.0,0.000000,...,2.000004,0.000000,0.0,0.0,0.000000,2.727724,0.000000,0.000000,2.160281,0.000000
3,2.198526,2.737880,1.989922,1.989922,4.761905,1.989922,4.890201,2.111002,0.0,0.000000,...,2.000004,0.000000,0.0,0.0,2.785353,0.000000,0.000000,0.000000,0.000000,2.103632
4,3.259272,2.395645,2.581268,2.581268,4.761905,2.581268,3.085484,2.111002,0.0,0.000000,...,2.000004,0.000000,0.0,0.0,0.000000,0.000000,2.687589,0.000000,0.000000,2.103632


In [83]:
print(preprocessed_data.shape)

(1000, 26)


* Split data in training and testing

In [87]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, y, random_state=42)

### MODELING

In [122]:
# Initialize the models
models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Ridge":Ridge(alpha=0.1) # con penalizacion
}

In [123]:
def evaluate_models(X_train, y_train, X_test, y_test, models):
    report = {}

    for i in range(len(list(models))):
        model = list(models.values())[i] # Retrieve the model

        # -------------------------------------------------------
        model.fit(X_train,y_train) 
        # -------------------------------------------------------
        # Get the predictions
        y_train_pred = model.predict(X_train) # train
        y_test_pred = model.predict(X_test) # test
        # Get the scores
        train_model_score = r2_score(y_train, y_train_pred)
        test_model_score = r2_score(y_test, y_test_pred)
        # Make the report
        report[list(models.keys())[i]] = test_model_score


        # -------------------------------------------------------
        mae = mean_absolute_error(y_train, y_train_pred)
        mse = mean_squared_error(y_train, y_train_pred)
        rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        r2_square = r2_score(y_train, y_train_pred)

        print(f'Model {list(models.keys())[i]}')
        print("- Root Mean Squared Error: {:.4f}".format(rmse))
        print("- MSE: {:.4f}".format(mse))
        print("- Mean Absolute Error: {:.4f}".format(mae))
        print("- R2 Score: {:.4f}".format(r2_square))
        
    return report

### EVALUATION

In [124]:
# Evaluate the models
model_report:dict=evaluate_models(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,
                                    models=models)

Model Random Forest
- Root Mean Squared Error: 0.6376
- MSE: 0.4065
- Mean Absolute Error: 0.3376
- R2 Score: 1.0000
Model Decision Tree
- Root Mean Squared Error: 0.0000
- MSE: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
Model Gradient Boosting
- Root Mean Squared Error: 1.4654
- MSE: 2.1473
- Mean Absolute Error: 1.0653
- R2 Score: 1.0000
Model Linear Regression
- Root Mean Squared Error: 0.0000
- MSE: 0.0000
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
Model XGBRegressor
- Root Mean Squared Error: 0.2193
- MSE: 0.0481
- Mean Absolute Error: 0.1514
- R2 Score: 1.0000
Model CatBoosting Regressor
- Root Mean Squared Error: 1.3476
- MSE: 1.8159
- Mean Absolute Error: 1.0336
- R2 Score: 1.0000
Model AdaBoost Regressor
- Root Mean Squared Error: 17.4272
- MSE: 303.7057
- Mean Absolute Error: 14.2481
- R2 Score: 0.9947
Model Ridge
- Root Mean Squared Error: 0.0331
- MSE: 0.0011
- Mean Absolute Error: 0.0253
- R2 Score: 1.0000


In [125]:
df_scores = pd.DataFrame(list(model_report.items()), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
df_scores

,Model Name,R2_Score
3,Linear Regression,1.000000
7,Ridge,1.000000
2,Gradient Boosting,0.999936
0,Random Forest,0.999936
1,Decision Tree,0.999914
4,XGBRegressor,0.999857
5,CatBoosting Regressor,0.999657
6,AdaBoost Regressor,0.995406


### TUNING OF HIPERPARAMETERS

In [131]:
# Initialize the models
models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}

In [132]:
# Hyperparameter Tunning
params={
    "Decision Tree": {
        'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        # 'splitter':['best','random'],
        # 'max_features':['sqrt','log2'],
    },
    "Random Forest":{
        # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        
        # 'max_features':['sqrt','log2',None],
        'n_estimators': [8,16,32,64,128,256]
    },
    "Gradient Boosting":{
        # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
        'learning_rate':[.1,.01,.05,.001],
        'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
        # 'criterion':['squared_error', 'friedman_mse'],
        # 'max_features':['auto','sqrt','log2'],
        'n_estimators': [8,16,32,64,128,256]
    },
    "Linear Regression":{},
    "XGBRegressor":{
        'learning_rate':[.1,.01,.05,.001],
        'n_estimators': [8,16,32,64,128,256]
    },
    "CatBoosting Regressor":{
        'depth': [6,8,10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100]
    },
    "AdaBoost Regressor":{
        'learning_rate':[.1,.01,0.5,.001],
        # 'loss':['linear','square','exponential'],
        'n_estimators': [8,16,32,64,128,256]
    }   
}

In [135]:
def evaluate_models_tuning(X_train, y_train, X_test, y_test, models, param):
    report = {}

    for i in range(len(list(models))):
        model = list(models.values())[i] # Retrieve the model
        # -------------------------------------------------------
        # hyperparameters
        para=param[list(models.keys())[i]]
        gs = GridSearchCV(model,para,cv=3)
        gs.fit(X_train, y_train) # Fit the model
        # Retrieve the best params
        model.set_params(**gs.best_params_)
        
        # fit with the best params
        model.fit(X_train,y_train) 
        # -------------------------------------------------------
        # Get the predictions
        y_train_pred = model.predict(X_train) # train
        y_test_pred = model.predict(X_test) # test
        # Get the scores
        train_model_score = r2_score(y_train, y_train_pred)
        test_model_score = r2_score(y_test, y_test_pred)
        # Make the report
        report[list(models.keys())[i]] = test_model_score
        print(list(models.keys())[i])
    return report

In [136]:
# Evaluate the models
model_report:dict=evaluate_models_tuning(X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,
                                    models=models, param=params)

Random Forest
Decision Tree
Gradient Boosting
Linear Regression
XGBRegressor
CatBoosting Regressor
AdaBoost Regressor


In [137]:
df_scores2 = pd.DataFrame(list(model_report.items()), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
df_scores2

,Model Name,R2_Score
3,Linear Regression,1.000000
4,XGBRegressor,0.999977
2,Gradient Boosting,0.999968
0,Random Forest,0.999936
1,Decision Tree,0.999894
5,CatBoosting Regressor,0.999266
6,AdaBoost Regressor,0.996118


### Applying regularization

In [ ]:
# Crear un modelo de regresión lineal con regularización L2
model = Ridge(alpha=0.1)
# Crear un modelo de regresión logística con regularización L1
model = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

#### SAVE THE BEST MODEL

In [138]:
# Get the best model score from dictionary
best_model_score = max(sorted(model_report.values()))
# Get best model name from dictionary
index_best_model = list(model_report.values()).index(best_model_score)
best_model_name = list(model_report.keys())[index_best_model]
# Get the best model
best_model = models[best_model_name]

# limit the model score
threshold = 0.8 # limit
if best_model_score < threshold:
    print("No best model found!")
print(f"Best found model on both training and testing dataset {best_model_name}")


Best found model on both training and testing dataset Linear Regression


In [ ]:
import pickle

file_path = "../0.dataset/"
file_name = "model.pkl"

with open(file_path + file_name, "wb") as file_obj_out:
    pickle.dump(best_model, file_obj_out)

### Predict

In [144]:
df.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Payment,cogs,gross margin percentage,gross income,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,Ewallet,522.83,4.761905,26.1415,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,Cash,76.40,4.761905,3.8200,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,Credit card,324.31,4.761905,16.2155,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,Ewallet,465.76,4.761905,23.2880,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,Ewallet,604.17,4.761905,30.2085,5.3


In [142]:
# 0. Solicitar inputs para las variables
branch = input("Ingrese la sucursal: ")
city = input("Ingrese la ciudad: ")
customer_type = input("Ingrese el tipo de cliente: ")
gender = input("Ingrese el género: ")
product_line = input("Ingrese la línea de producto: ")
unit_price = float(input("Ingrese el precio unitario: "))
quantity = int(input("Ingrese la cantidad: "))
tax = float(input("Ingrese el impuesto (en porcentaje): "))
payment = input("Ingrese el método de pago: ")
cogs = float(input("Ingrese el costo de los bienes vendidos: "))
gross_margin_percentage = float(input("Ingrese el margen bruto (en porcentaje): "))
gross_income = float(input("Ingrese el ingreso bruto: "))
rating = float(input("Ingrese la calificación: "))

# 1. Crear un diccionario con los inputs ingresados:

inputs = {
    'Branch': branch,
    'City': city,
    'Customer type': customer_type,
    'Gender': gender,
    'Product line': product_line,
    'Unit price': unit_price,
    'Quantity': quantity,
    'Tax 5%': tax,
    'Payment': payment,
    'cogs': cogs,
    'gross margin percentage': gross_margin_percentage,
    'gross income': gross_income,
    'Rating': rating
}

# 2. Convertir el diccionario en un dataframe:
df_input = pd.DataFrame.from_dict([inputs])

# 3. Aplicar el preprocesador al dataframe:
preprocessed_data_input = preprocessor.transform(df_input)

# 3.1. opcional convertir a dataframe


# 4. Aplicar el modelo para predecir:
prediction = model.predict(preprocessed_data_input)

print("El total es: ", prediction)